### Get Model Path from MLRun DB

In [1]:
from mlrun import get_run_db
from mlrun.artifacts import get_model

# MLRun DB connection
db = get_run_db()

# Artifact search params
artifact_name = "model"
project = "getting-started-tutorial-anyingli"
tag = "latest"

# Query DB for latest model
latest_model = db.list_artifacts(name=artifact_name, project=project, tag=tag)[0]
print(f"Model DB Key : {latest_model['db_key']}")

# Build DB path for use in Nuclio function
model_path = f'store://artifacts/{project}/{latest_model.get("db_key")}:{latest_model.get("tree")}'
print(f"MLRun DB Path: {model_path}")

# Get model file itself for manually loading
model_file, model_artifact, extra_data = get_model(latest_model["target_path"], suffix='.pkl')
print(f"Model File: {model_file}")

Model DB Key : train-iris-train_iris_model
MLRun DB Path: store://artifacts/getting-started-tutorial-anyingli/train-iris-train_iris_model:145f7e48ff394a73977393873b3589ed
Model File: /v3io/projects/getting-started-tutorial-anyingli/artifacts/145f7e48ff394a73977393873b3589ed/models/model.pkl


### Run info from Model Artifact

In [2]:
model_artifact.metrics # or latest_model["metrics"]

{'accuracy': 0.6333333333333333,
 'test-error': 0.36666666666666664,
 'auc-micro': 0.6433333333333333,
 'auc-weighted': 0.8426900584795322,
 'f1-score': 0.5402298850574713,
 'precision_score': 0.48333333333333334,
 'recall_score': 0.6666666666666666}

In [3]:
model_artifact.labels # or latest_model["labels"]

{'class': 'logistic_regression',
 'othertag': 'test',
 'C': '0.0001',
 'penalty': 'l2',
 'maxiter': '1000'}

In [4]:
model_artifact.producer # or latest_model["producer"]

{'name': 'train-iris-train_iris',
 'kind': 'run',
 'uri': 'getting-started-tutorial-anyingli/145f7e48ff394a73977393873b3589ed',
 'owner': 'anyingli'}

### Use Model in Nuclio Function
From `demos/getting-started-tutorial/03-model-serving.ipynb`

In [5]:
from os import path
import mlrun

In [6]:
project_name, artifact_path = mlrun.set_environment(project=project)

In [7]:
# nuclio: start-code

In [8]:
from cloudpickle import load
import numpy as np
from typing import List
import mlrun

class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [9]:
# nuclio: end-code

In [10]:
from mlrun import code_to_function
from mlrun.platforms import auto_mount

serving_fn = code_to_function('serving', kind='serving',image='mlrun/mlrun')
serving_fn.spec.default_class = 'ClassifierModel'

print(model_path)
serving_fn.add_model('my_model',model_path=model_path)

serving_fn = serving_fn.apply(auto_mount())

store://artifacts/getting-started-tutorial-anyingli/train-iris-train_iris_model:145f7e48ff394a73977393873b3589ed


In [11]:
serving_fn.deploy()

> 2021-03-02 00:43:51,635 [info] Starting remote function deploy
2021-03-02 00:43:51  (info) Deploying function
2021-03-02 00:43:51  (info) Building
2021-03-02 00:43:51  (info) Staging files and preparing base images
2021-03-02 00:43:51  (info) Building processor image
2021-03-02 00:43:52  (info) Build complete
2021-03-02 00:43:58  (info) Function deploy complete
> 2021-03-02 00:43:59,542 [info] function deployed, address=default-tenant.app.roblox-demo.iguazio-c0.com:30440


'http://default-tenant.app.roblox-demo.iguazio-c0.com:30440'

In [12]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
serving_fn.invoke('/v2/models/my_model/infer', my_data)

{'id': '49ba73fb-35fb-4eb0-93fc-b6f37d20d6f4',
 'model_name': 'my_model',
 'outputs': [0, 1]}